In [46]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity


In [43]:
df = pd.read_csv('Dataset.csv', usecols=['StockCode', 'Description'])
df = df.dropna()
df = df.drop_duplicates()

In [44]:
# print n of unique StockCode
print("Number of unique StockCode: ", len(df['StockCode'].unique()))

# get duplicate StockCode
duplicateStockCode = df[df.duplicated(['StockCode'])]
print("Number of duplicate StockCode: ", len(duplicateStockCode))
# print the first 5 duplicate StockCode
print(duplicateStockCode.head())
# print all of StockCode 22632
print("All of StockCode 22632: ")
print(df[df['StockCode'] == '22778'])

# decide not to remove duplicate StockCode because they may have different description

Number of unique StockCode:  3958
Number of duplicate StockCode:  834
     StockCode                         Description
257      22632           HAND WARMER RED RETROSPOT
2343     22972              CHILDRENS SPACEBOY MUG
3305     22965  3 TRADITIONAl BISCUIT CUTTERS  SET
3414     22776        CAKESTAND, 3 TIER, LOVEHEART
3415     22778                GLASS BELL JAR SMALL
All of StockCode 22632: 
       StockCode           Description
138        22778    GLASS CLOCHE SMALL
3415       22778  GLASS BELL JAR SMALL
478932     22778                 check


In [45]:
item_tuples = [(row['StockCode'], row['Description']) for index, row in df.iterrows()]

# print the first 10 tuples
print(item_tuples[:10])

[('85123A', 'WHITE HANGING HEART T-LIGHT HOLDER'), ('71053', 'WHITE METAL LANTERN'), ('84406B', 'CREAM CUPID HEARTS COAT HANGER'), ('84029G', 'KNITTED UNION FLAG HOT WATER BOTTLE'), ('84029E', 'RED WOOLLY HOTTIE WHITE HEART.'), ('22752', 'SET 7 BABUSHKA NESTING BOXES'), ('21730', 'GLASS STAR FROSTED T-LIGHT HOLDER'), ('22633', 'HAND WARMER UNION JACK'), ('22632', 'HAND WARMER RED POLKA DOT'), ('84879', 'ASSORTED COLOUR BIRD ORNAMENT')]


In [47]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
def embed(input):
  return model(input)


def calculate_cosine_similarity(vec_a, vec_b):
    vec_a = np.array(vec_a).reshape(1, -1)
    vec_b = np.array(vec_b).reshape(1, -1)
    
    similarity = cosine_similarity(vec_a, vec_b)
    return similarity[0][0]

Cosine similarity:  0.63785994
